In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import json

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
stemmer = LancasterStemmer()

In [3]:
with open('intents.json', encoding='UTF8') as file:
    data = json.load(file)

In [4]:
data

{'intents': [{'tag': 'google',
   'patterns': ['google', 'search', 'internet'],
   'responses': ['Redirecting to Google...']},
  {'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day',
    'Namaste',
    'yo'],
   'responses': ['Hello',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Get lost',
    'Till next time',
    'bbye'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I u

In [5]:
words = []
labels = []
docs_x = []
docs_y = []

In [6]:
# append, extend 차이점
# append 는 변수 그자체, 각 항목들 
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [7]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [23]:
print(stemmer.stem(docs_x[0][0].lower()))

googl


In [8]:
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w.lower()) for w in doc]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
            
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)

In [9]:
training = np.array(training)
output = np.array(output)

In [10]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
model.fit(training, output, n_epoch=1500, batch_size=8, show_metric=True)

Training Step: 22499  | total loss: 0.00932 | time: 0.069s
| Adam | epoch: 1500 | loss: 0.00932 - acc: 0.9883 -- iter: 112/113
Training Step: 22500  | total loss: 0.00839 | time: 0.073s
| Adam | epoch: 1500 | loss: 0.00839 - acc: 0.9894 -- iter: 113/113
--


In [12]:
def bag_of_words(s, word):
    bag = [0 for _ in range(len(word))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(w.lower()) for w in s_words]
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)

In [13]:
def chat():
    print('Start talking with your bot(type quit to stop) !')
    while True:
        inp = input("You : ")
        if inp.lower() == "quit":
            break
            
        results = model.predict([bag_of_words(inp, words)])
        results_index = np.argmax(results)
        tag = labels[results_index]
        
        for tg in data['intents']:
            if tg['tag'] == tag:
                responses = tg['responses']
                
        print(random.choice(responses))

chat()

Start talking with your bot(type quit to stop) !
You : hi
Hi there, how can I help?
You : how are you doing
All good..What about you?
You : Thanks
Happy to help!
You : Where are you from
I am Ted, a Deep-Learning chatbot
You : what's your name
All good..What about you?
You : quit
